<h1>Energy Consumption Forecasting</h1>

Real-time daily data for Energy Met and Daily Maximum Load at the national level taken from Grid-India (Grid Controller of India Limited) National Despatch Centre.

Data source: https://report.grid-india.in/psp_report.php


<h2>Data Extraction</h2>

Using BeautifulSoup and requests libraries, downloading and saving daily data into folders

In [139]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import os


# Base URL of the website
base_url = 'https://report.grid-india.in'

# Function to generate the form data for a given date
def generate_form_data(date):
    return {
        'selected_date': date.strftime('%Y-%m-%d')
    }

# Function to download a file from a URL
def download_file(url, save_dir, file_name):
    response = requests.get(url)
    if response.status_code == 200:
        file_path = os.path.join(save_dir, file_name)
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {file_name}")
    else:
        print(f"Failed to download {file_name}")

# Function to extract and download PDF reports from the form submission response
def extract_and_download_pdfs(response_text, save_dir):
    soup = BeautifulSoup(response_text, 'html.parser')
    links = soup.find_all('a', href=True)
    for link in links:
        url = link['href']
        if url.endswith('.pdf'):  # Check if the link is for a PDF file
            if not url.startswith('http'):
                url = base_url + url[1:]  # Ensure the URL is absolute
            file_name = url.split('/')[-1]
            download_file(url, save_dir, file_name)

# Main function to download PDF reports over a date range
def download_reports(start_date, end_date, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    url = base_url + '/psp_report.php'  # Form submission URL
    current_date = start_date
    while current_date <= end_date:
        form_data = generate_form_data(current_date)
        response = requests.post(url, data=form_data)
        if response.status_code == 200:
            extract_and_download_pdfs(response.text, save_dir)
        else:
            print(f"Failed to fetch report for {current_date.strftime('%Y-%m-%d')}")
        current_date += timedelta(days=1)

# Example usage
start_date = datetime(2020, 1, 1)
end_date = datetime(2020, 12, 31)
save_dir = 'reports\\2020'

download_reports(start_date, end_date, save_dir)

Downloaded: 01.01.20_NLDC_PSP.pdf
Downloaded: 02.01.20_NLDC_PSP.pdf
Downloaded: 03.01.20_NLDC_PSP.pdf
Downloaded: 04.01.20_NLDC_PSP.pdf
Downloaded: 05.01.20_NLDC_PSP.pdf
Downloaded: 06.01.20_NLDC_PSP.pdf
Downloaded: 07.01.20_NLDC_PSP.pdf
Downloaded: 08.01.20_NLDC_PSP.pdf
Downloaded: 09.01.20_NLDC_PSP.pdf
Downloaded: 10.01.20_NLDC_PSP.pdf
Downloaded: 11.01.20_NLDC_PSP.pdf
Downloaded: 12.01.20_NLDC_PSP.pdf
Downloaded: 13.01.20_NLDC_PSP.pdf
Downloaded: 14.01.20_NLDC_PSP.pdf
Downloaded: 16.01.20_NLDC_PSP.pdf
Downloaded: 17.01.20_NLDC_PSP.pdf
Downloaded: 18.01.20_NLDC_PSP.pdf
Downloaded: 19.01.20_NLDC_PSP.pdf
Downloaded: 20.01.20_NLDC_PSP.pdf
Downloaded: 21.01.20_NLDC_PSP.pdf
Downloaded: 22.01.20_NLDC_PSP.pdf
Downloaded: 23.01.20_NLDC_PSP.pdf
Downloaded: 24.01.20_NLDC_PSP.pdf
Downloaded: 25.01.20_NLDC_PSP.pdf
Downloaded: 26.01.20_NLDC_PSP.pdf
Downloaded: 27.01.20_NLDC_PSP.pdf
Downloaded: 28.01.20_NLDC_PSP.pdf
Downloaded: 29.01.20_NLDC_PSP.pdf
Downloaded: 30.01.20_NLDC_PSP.pdf
Downloaded: 31

Iterating through the folder containing the PDF files, extracting the required data from the files and writing it into csv files for later use.

I'm doing this year-wise in order to incorporate minor changes in the format of the PDF files.

In [29]:
import os
import pandas as pd
from tabula import read_pdf
import warnings

count = 0

# Function to process a single PDF file
def process_pdf(pdf_path):
    try:

         # Suppress warnings from tabula
        warnings.filterwarnings("ignore", category=UserWarning, message=".*Fallback font.*")

        # Extract date from the file path
        parts = pdf_path.split("\\")
        date_part = parts[-1].split("_")[0]
        day, month, year = map(int, date_part.split("."))
        date = pd.to_datetime(f'{int(year) + 2000}-{month}-{day}')
        
        # Extract tables from page 2 
        tables = read_pdf(pdf_path, pages="2", multiple_tables=True)

        # Check if any tables were found
        if not tables:
            print(f"No tables found on page 2 of the PDF: {pdf_path}")
            return None, None

        df = tables[2]
        state_col = df.columns[1]
        max_demand_column = df.columns[3]
        energy_met_column = df.columns[5]
        df.rename(columns={state_col: 'State', max_demand_column: 'Max.Demand', energy_met_column: 'Energy Met'}, inplace=True)
        df = df[['State', 'Max.Demand', 'Energy Met']]

        # Find the index to start from (Punjab)
        index_to_keep_from = df[df['State'] == 'Punjab'].index[0]
        index_to_stop=df[df['State'] == 'Tripura'].index[0]
        df = df.loc[index_to_keep_from:index_to_stop]
        df.reset_index(drop=True, inplace=True)

        return date, df
       
    except Exception as e:
        print(f"Error processing file {pdf_path}: {e}")
        return None, None

# Folder containing PDF files
pdf_folder = "reports\\2020"

# Initialize lists to store data for energy_met and maximum_demand dataframes
energy_met_data = []
maximum_demand_data = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)
        date, df = process_pdf(pdf_path)
        if date is not None and df is not None:
            try:
                states = df['State'].tolist()
                energy_met_data.append([date] + df['Energy Met'].tolist())
                maximum_demand_data.append([date] + df['Max.Demand'].tolist())
                count += 1
                print(f"Dataframe appended: {date}   ...{count}")
                
            except Exception as e:
                print(f"Error appending data for date {date}: {e}")



Dataframe appended: 2020-03-22 00:00:00   ...1
Dataframe appended: 2020-03-23 00:00:00   ...2
Dataframe appended: 2020-03-24 00:00:00   ...3
Dataframe appended: 2020-03-28 00:00:00   ...4
Dataframe appended: 2020-03-29 00:00:00   ...5
Dataframe appended: 2020-03-30 00:00:00   ...6
Dataframe appended: 2020-03-31 00:00:00   ...7


In [30]:
for el in energy_met_data:
    if len(el)!=37:
        print(energy_met_data.index(el)+1, len(el))


print(len(states))

36


In [31]:
# Create energy_met and maximum_demand dataframes
energy_met_df = pd.DataFrame(energy_met_data,columns=['Date'] + states)
maximum_demand_df = pd.DataFrame(maximum_demand_data,columns=['Date'] + states)


# Display the final dataframes
print("Energy Met:\n", energy_met_df)
print("\nMaximum Demand:\n", maximum_demand_df)

Energy Met:
         Date Punjab Haryana Rajasthan Delhi     UP Uttarakhand    HP  \
0 2020-03-22  105.7   115.1     202.4  59.1  255.7        33.4  21.1   
1 2020-03-23   86.3    93.0     178.7  46.1  240.8        22.5  18.0   
2 2020-03-24   74.7    92.0     171.9  48.7  247.4        21.7  20.5   
3 2020-03-28   57.6    65.9     105.8  44.4  217.3        17.0  12.9   
4 2020-03-29   60.0    66.5     131.8  41.8  220.4        16.8  12.7   
5 2020-03-30   60.0    66.5     131.8  41.8  220.4        16.8  12.7   
6 2020-03-31   63.6    66.7     141.3  42.9  234.8        17.8  13.0   

  J&K(UT) & Ladakh(UT) Chandigarh  ... Odisha West Bengal Sikkim  \
0                 47.7        2.8  ...   71.6       135.2    1.4   
1                 45.6        2.4  ...   66.5       100.4    1.0   
2                 44.7        2.4  ...   69.1       114.7    1.4   
3                 42.9        2.4  ...   80.7       117.0    1.3   
4                 45.3        2.2  ...   80.1       125.7    1.1   
5 

In [32]:
energy_met_df.to_csv('reports\\new_csv\\energy_2020.csv', index=False,mode='a')
maximum_demand_df.to_csv('reports\\new_csv\\max_2020.csv', index=False, mode='a')

Troubleshoot and understand the format of erroneous PDF files, and then appending to the required csv file

In [26]:
import pandas as pd
from tabula import read_pdf

# Specify the PDF file path
pdf_path = "reports\\2020\\21.03.20_NLDC_PSP.pdf"

# Extract tables from page 2 
tables = read_pdf(pdf_path, pages="2", multiple_tables=True)

# Check if any tables were found
if not tables:
    print("No tables found on page of the PDF.")
else:
    # Assuming there's only one table on page 2 (adjust if needed)
    #print(tables)
    df = tables[2]

df.head()
df.to_csv('test.csv', index=False)

In [27]:
import warnings
# Function to process a single PDF file
def process_pdf(pdf_path):
    try:
        # Suppress warnings from tabula
        warnings.filterwarnings("ignore", category=UserWarning, message=".*Fallback font.*")

        # Extract tables from page 2 
        tables = read_pdf(pdf_path, pages="2", multiple_tables=True)

        # Check if any tables were found
        if not tables:
            print(f"No tables found on page 2 of the PDF: {pdf_path}")
            return None, None

        df = pd.read_csv('test.csv')
        #df = tables[2]
        state_col = df.columns[1]
        max_demand_column = df.columns[3]
        energy_met_column = df.columns[5]
        df.rename(columns={state_col: 'State', max_demand_column: 'Max.Demand',energy_met_column: 'Energy Met'}, inplace=True)
        #df[['Max.Demand', 'Extra']] = df['Max.Demand'].str.split(' ', n=1, expand=True)
        #df[['Energy Met', 'Extra2']] = df['Energy Met'].str.split(' ', n=1, expand=True)
        df = df[['State', 'Max.Demand', 'Energy Met']]

        # Find the index to start from (Punjab)
        #df=df.drop(df.index[18]) drop a column of specified index
        index_to_keep_from = df[df['State'] == 'Punjab'].index[0]
        index_to_stop=df[df['State'] == 'Tripura'].index[0]
        df = df.loc[index_to_keep_from:index_to_stop]
        df.reset_index(drop=True, inplace=True)
        df.to_csv('test.csv', index=False)
        #df=df.drop(df.index[13])
        #df=df.drop(df.index[26])
        df.reset_index(drop=True, inplace=True)
        df.to_csv('test.csv', index=False,mode='a')

        # Extract date from the file path
        parts = pdf_path.split("\\")
        date_part = parts[-1].split("_")[0]
        day, month, year = map(int, date_part.split("."))
        date = pd.to_datetime(f'{int(year) + 2000}-{month}-{day}')

        return date, df

    except Exception as e:
        print(f"Error processing file {pdf_path}: {e}")
        return None, None
 
# Path to the single PDF file
pdf_path = "reports\\2020\\21.03.20_NLDC_PSP.pdf"

# Initialize lists to store data for energy_met and maximum_demand dataframes
energy_met_data = []
maximum_demand_data = []

# Process the single PDF file
date, df = process_pdf(pdf_path)
if date is not None and df is not None:
    try:
        states = df['State'].tolist()
        energy_met_data.append([date] + df['Energy Met'].tolist())
        maximum_demand_data.append([date] + df['Max.Demand'].tolist())
        print(f"Dataframe appended: {date}")
    except Exception as e:
        print(f"Error appending data for date {date}: {e}")


energy_met_df = pd.DataFrame(energy_met_data, columns=['Date'] + states)
maximum_demand_df = pd.DataFrame(maximum_demand_data, columns=['Date'] + states)


# Create energy_met and maximum_demand dataframes
if energy_met_data and maximum_demand_data:
    energy_met_df = pd.DataFrame(energy_met_data, columns=['Date'] + states)
    maximum_demand_df = pd.DataFrame(maximum_demand_data, columns=['Date'] + states)

    # Append dataframes to CSV file
    energy_met_csv_path = "reports\\new_csv\\energy_2020.csv"
    maximum_demand_csv_path = "reports\\new_csv\\max_2020.csv"
    energy_met_df.to_csv(energy_met_csv_path, index=False, header=False,mode='a')
    maximum_demand_df.to_csv(maximum_demand_csv_path, index=False,header=False,mode='a')
    print(f"Dataframes appended to CSV files: {energy_met_csv_path}, {maximum_demand_csv_path}")
else:
    print("No data was processed.")


Dataframe appended: 2020-03-21 00:00:00
Dataframes appended to CSV files: reports\new_csv\energy_2020.csv, reports\new_csv\max_2020.csv


Consolidating individual csv files into 2 files
- energy_states_consolidated 
- max_states_consolidated

In [1]:
import pandas as pd
import os

# Directory containing the CSV files
directory = 'reports\\new_csv'

# List all CSV files in the directory
energy_csv_files = [file for file in os.listdir(directory) if file.endswith('.csv') and file.startswith('energy')]
max_csv_files = [file for file in os.listdir(directory) if file.endswith('.csv') and file.startswith('max')]

# Read each CSV file and concatenate them into a single DataFrame
energy_dfs = []
for file in energy_csv_files:
    df = pd.read_csv(os.path.join(directory, file))
    energy_dfs.append(df)

max_dfs = []
for file in max_csv_files:
    df = pd.read_csv(os.path.join(directory, file))
    max_dfs.append(df)

# Concatenate all DataFrames into one
energy_result = pd.concat(energy_dfs, ignore_index=True)
energy_result.reset_index(drop=True, inplace=True)
max_result = pd.concat(max_dfs, ignore_index=True)
max_result.reset_index(drop=True, inplace=True)

# Write the consolidated DataFrame to a new CSV file
energy_result.to_csv('energy_states_consolidated.csv', index=False)
max_result.to_csv('max_states_consolidated.csv', index=False)

print("Consolidated CSV file saved successfully.")


Consolidated CSV file saved successfully.
